# Programming Assignment 1
## Genre Classification using Locality Sensitive Hashing (LSH)


In [4]:
#Imports

import pandas as pd
import numpy as np

### Data Loading and Preprocessing 

In [10]:
# Load data
df_tracks = pd.read_csv('tracks.csv', index_col=0, header=[0, 1])
df_tracks = df_tracks[df_tracks['set']['subset'] == 'medium']
df_features = pd.read_csv('features.csv', index_col=0, header=[0, 1, 2])

# Filter by genres
df_tracks = df_tracks[df_tracks['track']['genre_top'].isin(['Hip-Hop', 'Pop', 'Folk', 'Rock', 'Experimental', 'International', 'Electronic', 'Instrumental'])]

# Split df_tracks into training, testing, and validation sets
df_tracks_train = df_tracks[df_tracks.iloc[:, 30] == 'training']
df_tracks_test = df_tracks[df_tracks.iloc[:, 30] == 'test']
df_tracks_validation = df_tracks[df_tracks.iloc[:, 30] == 'validation']

# Match features with tracks for training, testing, and validation
df_features_train = df_features[df_features.index.isin(df_tracks_train.index)]
df_features_test = df_features[df_features.index.isin(df_tracks_test.index)]
df_features_validation = df_features[df_features.index.isin(df_tracks_validation.index)]

# Extract feature values
X_train = df_features_train.values
X_test = df_features_test.values
X_validation = df_features_validation.values

# Extract genre labels
y_train = df_tracks_train['track']['genre_top']
y_test = df_tracks_test['track']['genre_top']
y_validation = df_tracks_validation['track']['genre_top']

In [46]:
print(X_train.shape)

(11912, 518)


### Random Projection Matrix

In [5]:
# r_i = rowsize, r_j) = columsize
def generate_random_matrix(r_i, r_j):
    rij = np.random.choice([-1, 0, 1], size=(r_i, r_j), p=[1/6, 2/3, 1/6])
    return np.sqrt(3) * rij

### Hashtable generator function

We use the transpose of the Random Projection Matrix to reduce the dimensionality  and determine the orientation of each track's data relative to the hyperplanes by using the dot Product of the feature matrix and the transposed Random Projection Matrix. 
Then we use the binary representations of the orientations as a bucket and put in the tracks accordingly. 
$ \begin{cases} 
0 & \text{ if } x < 0 \\
1 & \text{ else}
\end{cases}
$ 
We can do this because of $\mathbf{a} \cdot \mathbf{b} = \|\mathbf{a}\| \|\mathbf{b}\| \cos(\theta)$ positive means on one side and negative on the other.
This whole process represents one hashtable.

In [68]:
"""
The binary representations are of length l.
And the number of hashtables we creat is equal to n.
"""
def hashtable_generator(X, l=32, n=20):
    hash_tables_and_matrices = []  # Will store tuples of (hash_table, random_matrix)
    for _ in range(n):
        buckets = {}
        random_matrix = generate_random_matrix(l, X.shape[1])
        X_dot = np.dot(X, random_matrix.T)
        X_dot = X_dot > 0
        X_dot = X_dot.astype(int)

        for i in range(len(X_dot)):
            hash_str = ''.join(X_dot[i].astype(str))
            if hash_str not in buckets:
                buckets[hash_str] = []
            buckets[hash_str].append(i)
        
        # Append both the hash table (buckets) and its corresponding random matrix
        hash_tables_and_matrices.append((buckets, random_matrix))
    
    return hash_tables_and_matrices


In [69]:
hash_tables_and_matrices = hashtable_generator(X_train)

In [70]:
def find_matching_songs(song_input, hash_tables_and_matrices, l=32):
    matching_songs_indices = set()

    for buckets, random_matrix in hash_tables_and_matrices:
        song_projected = np.dot(song_input, random_matrix.T) > 0
        song_hash = ''.join(song_projected.astype(int).astype(str))

        if song_hash in buckets:
            matching_songs_indices.update(buckets[song_hash])

    return list(matching_songs_indices)

In [71]:
matches =  find_matching_songs(X_test[1],hash_tables_and_matrices)

In [74]:
def compute_distances(X, song_input, matching_songs, metric="euclid", cut=10):
    filtered_songs = []
    if metric == "euclid":
        for element in matching_songs:
            distance = np.linalg.norm(X[element] - song_input)
            filtered_songs.append((element, distance))
    elif metric == "cosine":
        for element in matching_songs:
            # Calculate cosine similarity
            dot_product = np.dot(X[element], song_input)
            norm_song = np.linalg.norm(X[element])
            norm_input = np.linalg.norm(song_input)
            similarity = dot_product / (norm_song * norm_input)
            
            # Convert similarity to distance (cosine distance)
            distance = 1 - similarity
            filtered_songs.append((element, distance))
    else:
        raise ValueError("Invalid metric specified. Use 'euclid' or 'cosine'.")
    
    sorted_songs = sorted(filtered_songs, key=lambda x: x[1])
    if cut is not None:
        sorted_songs = sorted_songs[:cut]
    
    return sorted_songs

In [75]:
compute_distances(X_train, X_test[1], matches, "cosine")

[(10209, 0.0005600534859354633),
 (4622, 0.0006203843290413236),
 (10446, 0.0006389835819047285),
 (3310, 0.0006580797243186387),
 (10782, 0.0007022478358682527),
 (6525, 0.0007095056876785799),
 (2661, 0.0007994446981818282),
 (2346, 0.0008073112474676902),
 (6287, 0.0008073265005050789),
 (1084, 0.0008350759096842353)]